# Organize directories

The first step in training a ML model on sequences of images acquired from the ProtectCode app is to ensure the directory structure is set up for subsequent data loading. 

These commands assume the following: 

1) That the top directory for the training data and this notebook are both located in ~/src/YPB-AI/ on the EC2 instance. To get a zipped directory from S3 to that directory on your EC2 instance, you can run the following bash command: "aws s3 cp s3://your-bucket-name/yourZipFile.zip ~/src/YPB-AI/yourZipFile.zip". And to unzip it, you can run "unzip yourZipFile.zip"

2) That the jupyter notebook server was opened from the ~/src/ directory on the EC2 instance via the command "jupyter notebook --ip=0.0.0.0 --no-browser".

3) That underneath the top directory (which I usually call "Training" or something of the like), there are a series of subdirectories named and organized by classification label. For instance, each subdirectory that contains true negative image sequences, according to my implementation, contains the word "Negative". And subdirectories with true positives do not contain the word "Negative". 



In [1]:
import numpy as np
from glob import glob
import os, sys
import re

Set the name of your top data directory below. 

In [2]:
# topDirName = 'Training'
# topDirName = 'Training2'
topDirName = 'Bam1-CIFAR'

Lump together all subdirectories with the same classification label. 

In [3]:
#Assumes current working directory is ~/src/YPB-AI
allSamples = np.array(glob(topDirName + '/*'))
os.system('mkdir ' + topDirName + '/Absent')
os.system('mkdir ' + topDirName + '/Present')

for sample in allSamples: 
    if 'Negative' in sample or 'Sat' in sample:
        cmdString = "mv " + sample + " " + topDirName + "/Absent"
    else:
        cmdString = "mv " + sample + " " + topDirName + "/Present"
    os.system(cmdString)

Each directory within the classification subdirectory contains a series of folders that have the following contents:

1) A .txt file containing some timing and other metadata about the app run.

2) A folder called 'Photos' which contains the sequence of images acquired.

The next cell will obliterate the middle directory (the directory just below the classification directory) and repackage each run's photos into a uniquely identifiable folder of its own (lacking the text file) at the level where the middle directories once sat.



In [4]:
# Find 'vestigial' directories to be deleted after unpacking.
presentVestDirs = np.array(glob(topDirName + '/Present/*'))
absentVestDirs = np.array(glob(topDirName + '/Absent/*'))

# Unpack the JPG photo files into identifiable directories.
allPresentSamples = np.array(glob(topDirName + '/Present/*/*'))
for sample in allPresentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    
allAbsentSamples = np.array(glob(topDirName + '/Absent/*/*'))
for sample in allAbsentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    
    
# Delete vestigial directories.
for direc in presentVestDirs:
    os.system('rm -rf ' + direc)
    
for direc in absentVestDirs:
    os.system('rm -rf ' + direc)

Look through all directories and find image sequences containing the file "IMG_9999.JPG". If found, sort and rename to an appropriate image sequence. This is necessary because when iOS reaches "IMG_9999.JPG" it wraps back around to "IMG_0000.JPG" for the next in the sequence. If this is left as is, the sorting operation that follows will sort the images into the incorrect timing order. 

In [5]:
# Get the filenames.
imgSeqsToCorrect = os.popen('cd ~/src/YPB-AI/; find -name "*IMG_9999.JPG"').read()
# Split into unique entities.
imgSeqsToCorrect = re.split('\n',imgSeqsToCorrect)
# Drop the last entry, it's empty.
imgSeqsToCorrect.pop();

# Change any files that don't start with 'IMG_9' (i.e., any files that come after the wraparound) to start with 'J'.
# This will mean that when they are sorted, they will come correctly after the wraparound.
for sample in imgSeqsToCorrect:
    digestedSample = re.split('/',sample)
    pathToDir = os.path.join(digestedSample[0],digestedSample[1],digestedSample[2],digestedSample[3])
    filesInDir = os.listdir(pathToDir)
    for i in range(len(filesInDir)):
        if 'IMG_9' not in filesInDir[i]:
            newFile = filesInDir[i].replace('I','J',1)
            cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
            os.system(cmdString)

# Change the images in the folder into a stack of 12-image input

In [6]:
import torch
import torchvision.models as models
from PIL import Image
import pickle

In [7]:
topdir = topDirName
ndx = 0
topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']

# Calculate the total number of stacks to transform.
numStacks = 0
for classDir in topdirs:
    thisClassDir = os.path.join(topdir,classDir)
    directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
    numStacks += len(directories)

for classDir in topdirs:
    thisClassDir = os.path.join(topdir,classDir)
    directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']


    # Do the transformation.
    for photoDir in directories:

        thisPhotoDir = os.path.join(thisClassDir, photoDir)
        ##pin pin fix##

#             print(thisPhotoDir)
#             print(os.path.isfile(photoDir))
#             continue

        if not os.path.isdir(thisPhotoDir):
            continue

        ## end ##
        files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
        print("Transforming image stack {}".format(thisPhotoDir))
        files.sort()
        output = []
        # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
        for file in files:
            thisFile = os.path.join(thisPhotoDir,file)
            imgArr = np.float32(np.array(Image.open(thisFile).resize((32,32))).reshape([1,3,32,32]))
            imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
            imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
            imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
            output.append(imgArr)

        FinalOutput = torch.FloatTensor(output)
        # for i in range(len(files)):
        #    FinalOutput[i] = output[i]
        pathStr = thisPhotoDir + '.pickle'
        with open(pathStr,'wb') as f:
            pickle.dump(FinalOutput,f)
            ndx += 1
            print("{}% complete.".format(ndx/numStacks*100))
    

Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-47-57
0.12376237623762376% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-00-08
0.24752475247524752% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_H_Negative_29-Oct-2019_17-03-27
0.3712871287128713% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-47-29
0.49504950495049505% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_V_Negative_30-Oct-2019_16-54-21
0.6188118811881188% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-46-52
0.7425742574257426% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-44-25
0.8663366336633664% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-42-37
0.9900990099009901% complete.
Transforming image stack Bam1-CIFAR/Absent/6

8.787128712871286% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_12-37-30
8.91089108910891% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_13-24-12
9.034653465346535% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_10-06-49
9.158415841584159% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_H_Negative_29-Oct-2019_16-55-30
9.282178217821782% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-42-56
9.405940594059405% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-33-16
9.52970297029703% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_H_Negative_29-Oct-2019_16-39-48
9.653465346534654% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_H_Negative_29-Oct-2019_17-12-27
9.777227722772277% complete.
Transforming image stack Bam1-C

17.326732673267326% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-14-06
17.450495049504948% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_10-11-49
17.574257425742573% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-14-36
17.698019801980198% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-21-10
17.82178217821782% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-53-10
17.945544554455445% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_10-10-35
18.06930693069307% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_V_Negative_30-Oct-2019_16-01-36
18.193069306930692% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-47-26
18.316831683168317% complete.
Transforming 

25.866336633663366% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_V_Negative_30-Oct-2019_17-22-21
25.99009900990099% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-50-33
26.113861386138616% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_H_Negative_29-Oct-2019_16-38-44
26.237623762376238% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_H_Negative_29-Oct-2019_16-57-37
26.361386138613863% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_V_Negative_30-Oct-2019_16-02-36
26.48514851485149% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_V_Negative_30-Oct-2019_16-45-59
26.60891089108911% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_13-23-25
26.732673267326735% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_V_Negative_30-Oct-2019_17-23-34
26.856435643564357% complete.
Transforming i

34.4059405940594% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_12-35-12
34.52970297029702% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_12-00-19
34.65346534653465% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_09-57-36
34.777227722772274% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_H_Negative_29-Oct-2019_17-05-04
34.900990099009896% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_H_Negative_29-Oct-2019_17-17-54
35.024752475247524% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-50-50
35.148514851485146% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_redBottle_V_Negative_30-Oct-2019_10-11-01
35.27227722772277% complete.
Transforming image stack Bam1-CIFAR/Absent/6S_filledBottle_H_Negative_30-Oct-2019_13-24-59
35.396039603960396% complete.
Transforming image sta

...

61.01485148514851% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-10-13
61.13861386138614% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_H_29-Oct-2019_15-34-26
61.26237623762376% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_H_29-Oct-2019_15-41-03
61.386138613861384% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_09-05-14
61.50990099009901% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_V_30-Oct-2019_10-59-43
61.633663366336634% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_H_29-Oct-2019_16-03-43
61.757425742574256% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-40-22
61.88118811881188% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_H_29-Oct-2019_16-19-57
62.004950495049506% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_V_30-Oct-2019_11-19-

70.42079207920791% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_09-30-19
70.54455445544554% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_H_29-Oct-2019_15-52-01
70.66831683168317% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-03-46
70.79207920792079% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-09-22
70.91584158415841% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_H_29-Oct-2019_15-48-49
71.03960396039604% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_V_30-Oct-2019_11-29-15
71.16336633663366% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_09-10-42
71.28712871287128% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-36-17
71.4108910891089% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_09-24-37
71.53465346

79.82673267326733% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-17-03
79.95049504950495% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_V_30-Oct-2019_11-04-26
80.07425742574257% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-43-53
80.19801980198021% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_08-53-44
80.32178217821783% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_H_29-Oct-2019_16-04-11
80.44554455445545% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_08-50-14
80.56930693069307% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_H_29-Oct-2019_15-57-07
80.6930693069307% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_11-02-49
80.81683168316832% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_V_30-Oct-2019_10-50-23
80.94059

89.23267326732673% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_09-22-42
89.35643564356435% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_H_29-Oct-2019_15-36-03
89.48019801980197% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_H_29-Oct-2019_15-50-43
89.60396039603961% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_V_30-Oct-2019_11-16-08
89.72772277227723% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_08-56-27
89.85148514851485% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_09-08-05
89.97524752475248% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_08-56-49
90.0990099009901% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-57-58
90.22277227722772% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_09-18-37
90.34653465

98.63861386138613% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_09-13-54
98.76237623762376% complete.
Transforming image stack Bam1-CIFAR/Present/6S_filledBottle_H_29-Oct-2019_15-56-12
98.88613861386139% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_09-28-16
99.00990099009901% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-16-36
99.13366336633663% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-54-08
99.25742574257426% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_09-00-31
99.38118811881188% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_V_30-Oct-2019_08-45-30
99.5049504950495% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-10-37
99.62871287128714% complete.
Transforming image stack Bam1-CIFAR/Present/6S_redBottle_H_29-Oct-2019_10-56-07
99.75247524752476

Use a pretrained 11-layer VGG with the final classifier layer chopped off. >> no not using this

In [7]:
# Import the CNN model and move it to CUDA if available.
model = models.vgg(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
del model.classifier

if torch.cuda.is_available():
    model.to('cuda');

Define the function that will transform model output into a pickle file.

In [8]:
def transform_to_pickle(model,topdir):
    model.eval()
    ndx = 0
    topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']
    
    # Calculate the total number of stacks to transform.
    numStacks = 0
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        numStacks += len(directories)
    
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        
        
        # Do the transformation.
        for photoDir in directories:
            
            thisPhotoDir = os.path.join(thisClassDir, photoDir)
            ##pin pin fix##
            
#             print(thisPhotoDir)
#             print(os.path.isfile(photoDir))
#             continue
            
            if not os.path.isdir(thisPhotoDir):
                continue

            ## end ##
            files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
            print("Transforming image stack {}".format(thisPhotoDir))
            files.sort()
            output = []
            # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
            for file in files:
                thisFile = os.path.join(thisPhotoDir,file)
                imgArr = np.float32(np.array(Image.open(thisFile).resize((224,224))).reshape([1,3,224,224]))
                imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
                imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
                imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
                if torch.cuda.is_available():
                    output.append(model(torch.cuda.FloatTensor(imgArr)))
                else:
                    output.append(model(torch.FloatTensor(imgArr)))

            FinalOutput = imgArr
            pathStr = thisPhotoDir + '.pickle'
            with open(pathStr,'wb') as f:
                pickle.dump(FinalOutput,f)
                ndx += 1
                print("{}% complete.".format(ndx/numStacks*100))
    

Do the transformation

In [9]:
transform_to_pickle(model,topDirName)

NameError: name 'model' is not defined

Clean up the directory, removing everything but the pickle files.

In [10]:
# Assemble all the pickle files and remove the rest.
classDirs = [direc for direc in os.listdir(topDirName) if direc[0] is not '.']
for classDir in classDirs:
    newDir = os.path.join(topDirName,classDir)
    fileDirs = [direc for direc in os.listdir(newDir) if direc[0] is not '.' and not direc.lower().endswith('.pickle')]
    for fileDir in fileDirs:
        pickleFileDir = os.path.join(newDir,fileDir)
        os.system('rm -rf ' + pickleFileDir)

# Split the directory into train/test/validation sets

In [11]:
import random

In [14]:
finalDirName = 'Split_Data_Bam1_CIFAR'

In [13]:
def split_folders(inputDir, outputDir, trainValTestRatio, seed):
    # Assume inputDir is organized as 'inputDir/class/topFolder/photos'
    classes = os.listdir(inputDir)
    # Exclude folders where '.' is the first character.
    classes = [directory for directory in classes if directory[0] != '.']
    numClasses = len(classes)
    # Create output directories
    os.system('mkdir ' + outputDir)
    os.system('mkdir ' + outputDir + '/train')
    os.system('mkdir ' + outputDir + '/valid')
    os.system('mkdir ' + outputDir + '/test')
    # Pull files and put into the appropriate folders.
    for directory in classes:
        files = os.listdir(os.path.join(inputDir, directory))
        files.sort()
        random.seed(seed)
        random.shuffle(files)
        numFiles = len(files)
        # Get the indices.
        ndxAmts = np.intc((np.round(trainValTestRatio / np.sum(trainValTestRatio) * numFiles)))
        numNdxes = np.sum(ndxAmts)
        if numNdxes != numFiles:
            ndxAmts[0] += numFiles-numNdxes
        trainFiles = files[0:ndxAmts[0]]
        valFiles = files[ndxAmts[0]:ndxAmts[0]+ndxAmts[1]]
        testFiles = files[ndxAmts[0]+ndxAmts[1]:]
        # Put the training directories in the appropriate folder
        # First make the class directories.
        os.system('mkdir ' + outputDir + '/train/' + directory)
        os.system('mkdir ' + outputDir + '/valid/' + directory)
        os.system('mkdir ' + outputDir + '/test/' + directory)
        for file in trainFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/train/' + directory + '/' + file)
        for file in valFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/valid/' + directory + '/' + file)
        for file in testFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/test/' + directory + '/' + file)

In [12]:
for idx in range(5):
    split_folders(topDirName, finalDirName_root + str(idx),[0.8,0.1,0.1],1000 + idx)

In [15]:
split_folders(topDirName, finalDirName,[0.8,0.1,0.1],7778)